Zdroje:
- https://www.tensorflow.org/model_optimization/guide/quantization/training_example

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [11]:
t = -3
np.angle(t)

3.141592653589793

In [3]:
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense

In [4]:
from keras.datasets import mnist

(inputs_train, labels_train), (inputs_test, labels_test) = mnist.load_data()

In [5]:
# Normalize inputs
inputs_train = inputs_train / 255
inputs_test = inputs_test / 255

In [136]:
# Create model
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    Conv2D(filters=16, kernel_size=(3, 3), activation=keras.activations.relu),
    MaxPool2D(),
    Conv2D(filters=32, kernel_size=(2, 2), activation=keras.activations.relu),
    MaxPool2D(),
    Flatten(),
    Dense(units = 10, activation=keras.activations.softmax),
])

In [159]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 16)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 32)        2080      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1152)             

In [137]:
# Compile model with adam optimizer and for accuracy
model.compile(optimizer="adam", loss=keras.losses.sparse_categorical_crossentropy, metrics=["accuracy"])

In [139]:
# Train model
model.fit(inputs_train, labels_train, batch_size=32, epochs=3, verbose=1, validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 18s 10ms/step - loss: 0.2244 - accuracy: 0.9365 - val_loss: 0.0681 - val_accuracy: 0.9803
Epoch 2/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0733 - accuracy: 0.9777 - val_loss: 0.0562 - val_accuracy: 0.9863
Epoch 3/3
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0563 - accuracy: 0.9830 - val_loss: 0.0542 - val_accuracy: 0.9847


In [140]:
model.evaluate(inputs_test, labels_test)

313/313 [==============================] - 1s 5ms/step - loss: 0.0481 - accuracy: 0.9837


[0.048064373433589935, 0.9836999773979187]

In [1]:
import quantization

In [158]:
annotated_model = quantization.quantize_model(model, [7, 7, 3])
quant_aware_model = quantization.apply_custom_quantization(annotated_model)
quant_aware_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_23 (Quantize  (None, 28, 28)           3         
 Layer)                                                          
                                                                 
 quant_reshape_1 (QuantizeWr  (None, 28, 28, 1)        1         
 apperV2)                                                        
                                                                 
 quant_conv2d_2 (QuantizeWra  (None, 26, 26, 16)       165       
 pperV2)                                                         
                                                                 
 quant_max_pooling2d_2 (Quan  (None, 13, 13, 16)       1         
 tizeWrapperV2)                                                  
                                                                 
 quant_conv2d_3 (QuantizeWra  (None, 12, 12, 32)      

In [155]:
# Compile quantized model
quant_aware_model.compile(optimizer="adam", loss=keras.losses.sparse_categorical_crossentropy, metrics=["accuracy"])

In [156]:
# Fine-tune quantized model
quant_aware_model.fit(inputs_train[0:500], labels_train[0:500], batch_size=16, epochs=1, verbose=1, validation_split=0.1)

57/57 [==============================] - 2s 13ms/step - loss: 0.0787 - accuracy: 0.9722 - val_loss: 0.0717 - val_accuracy: 0.9900


In [157]:
quant_aware_model.evaluate(inputs_test, labels_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0567 - accuracy: 0.9830


[0.05669531971216202, 0.9829999804496765]